In [4]:
import pandas as pd
!pip install -U pandasql
from pandasql import sqldf

In [5]:
mysql = lambda q: sqldf(q, globals())
products= pd.read_csv('Amazon_Products.csv', low_memory=False)

In [6]:
prod_clean = products.loc[:, ~products.columns.str.contains('^Unnamed')]

In [7]:
### base of all analysis ~
cleaned_base = """
SELECT * FROM (
SELECT uniq_id
, product_name
, manufacturer
, price
, coalesce(cast(number_available_in_stock as int),0) number_available_in_stock
, coalesce(cast(number_of_reviews as int),0) number_of_reviews
, coalesce(cast(number_of_answered_questions as int),0) number_of_answered_questions
, cast(substr(average_review_rating,1,3) as float) avg_review_rating_num
, amazon_category_and_sub_category
, description
, product_information
, product_description
, items_customers_buy_after_viewing_this_item
, customer_questions_and_answers
FROM prod_clean)
WHERE number_of_reviews > 10
AND uniq_id is not null 
AND length(uniq_id) > 1
"""
#category high level stats
category = """
SELECT distinct SUBSTR(category_and_sub_category,1,INSTR(category_and_sub_category,'>') -1) category_lvl_1
, avg(avg_review_rating_num) avg_rating
, avg(number_of_reviews) avg_reviews
, avg(number_of_answered_questions) avg_questions_answered
, COUNT(*) count_of_products 
, DENSE_RANK() over (ORDER BY  avg(number_of_reviews) DESC, count(*) DESC, avg(avg_review_rating_num) DESC) as popularity_rank
FROM (
SELECT uniq_id
, product_name
, manufacturer
, coalesce(nullif(trim(price),''),0) item_price
, coalesce(cast(number_available_in_stock as int),0) number_available_in_stock
, coalesce(cast(number_of_reviews as int),0) number_of_reviews
, coalesce(cast(number_of_answered_questions as int),0) number_of_answered_questions
, cast(substr(average_review_rating,1,3) as float8) avg_review_rating_num
, coalesce(amazon_category_and_sub_category, 'None') category_and_sub_category
, description
, product_information
, product_description
, items_customers_buy_after_viewing_this_item
, customer_questions_and_answers
FROM prod_clean
WHERE uniq_id is not null 
AND length(uniq_id) > 1)
GROUP BY 1
ORDER BY popularity_rank ASC
"""

# DISTINCT CASE
# WHEN stock_level = 0 THEN 'Out of Stock' 
# WHEN stock_level BETWEEN 1 AND 15 THEN 'Low Stock'
# WHEN stock_level BETWEEN 16 AND 30 THEN 'Medium Stock'
# WHEN stock_level BETWEEN 30 AND 45 THEN 'High Stock'
# WHEN stock_level  > 46 THEN 'Very High Stock'
# END AS current_stock

# understanding product pricing
prod= """
SELECT
DISTINCT CASE
WHEN stock_level = 0 THEN 'Out of Stock' 
WHEN stock_level BETWEEN 1 AND 15 THEN 'Low Stock'
WHEN stock_level BETWEEN 16 AND 30 THEN 'Medium Stock'
WHEN stock_level BETWEEN 30 AND 45 THEN 'High Stock'
WHEN stock_level  > 46 THEN 'Very High Stock'
END AS current_stock
,SUM(CASE WHEN item_price = 0 THEN 1 ELSE 0 END) AS free_item
, SUM(CASE WHEN item_price BETWEEN 1 AND 20 THEN 1 ELSE 0 END) AS price_0_TO_20
, SUM(CASE WHEN item_price BETWEEN 21 AND 40 THEN 1 ELSE 0 END) AS price_21_TO_40
, SUM(CASE WHEN item_price BETWEEN 41 AND 60 THEN 1 ELSE 0 END) AS price_41_TO_60
, SUM(CASE WHEN item_price BETWEEN 61 AND 80 THEN 1 ELSE 0 END) AS price_61_TO_80
, SUM(CASE WHEN item_price BETWEEN 81 AND 100 THEN 1 ELSE 0 END) AS price_81_TO_100
, SUM(CASE WHEN item_price BETWEEN 101 AND 120 THEN 1 ELSE 0 END) AS price_101_TO_120
, SUM(CASE WHEN item_price BETWEEN 121 AND 140 THEN 1 ELSE 0 END) AS price_121_TO_140
, SUM(CASE WHEN item_price BETWEEN 141 AND 160 THEN 1 ELSE 0 END) AS price_141_TO_160
, SUM(CASE WHEN item_price > 160 THEN 1 ELSE 0 END) AS price_160_plus
FROM (
SELECT uniq_id
, product_name
, manufacturer
, coalesce(cast(nullif(substr(price,2,length(price)),'') as float8),0) item_price
, coalesce(cast(number_available_in_stock as int),0) stock_level
, coalesce(cast(number_of_reviews as int),0) number_of_reviews
, coalesce(cast(number_of_answered_questions as int),0) number_of_answered_questions
, cast(substr(average_review_rating,1,3) as float8) avg_review_rating_num
, coalesce(amazon_category_and_sub_category, 'None') category_and_sub_category
, description
, product_information
, product_description
, items_customers_buy_after_viewing_this_item
, customer_questions_and_answers
FROM prod_clean
WHERE uniq_id is not null 
AND length(uniq_id) > 1)
GROUP BY 1
"""


In [49]:
mysql(prod)

,current_stock,free_item,price_0_TO_20,price_21_TO_40,price_41_TO_60,price_61_TO_80,price_81_TO_100,price_101_TO_120,price_121_TO_140,price_141_TO_160,price_160_plus
0,None,0,5,0,0,0,0,0,0,0,0
1,High Stock,9,129,27,15,2,0,0,0,0,0
2,Low Stock,1125,3988,764,227,83,48,27,23,13,65
3,Medium Stock,29,530,123,35,21,8,3,1,3,6
4,Out of Stock,271,1656,283,73,30,26,13,12,8,40
5,Very High Stock,1,36,9,4,3,1,0,0,0,0


In [8]:
mysql(category)

,category_lvl_1,avg_rating,avg_reviews,avg_questions_answered,count_of_products,popularity_rank
0,"Storage, Cleaning & Ring Sizers",4.500000,802.000000,23.000000,1,1
1,Bedding & Linens,4.100000,299.000000,39.000000,1,2
2,Camping & Hiking,4.200000,77.000000,11.000000,1,3
3,"Sweets, Chocolate & Gum",4.450000,64.500000,1.500000,2,4
4,"Pens, Pencils & Writing Supplies",4.385714,25.142857,2.428571,7,5
5,Pretend Play,4.664444,23.355556,2.444444,45,6
6,Men,4.100000,18.000000,3.000000,1,7
7,Puppets & Puppet Theatres,4.729740,14.973978,1.152416,269,8
8,Sports Toys & Outdoor,4.577419,14.924731,2.212366,372,9
9,Musical Toy Instruments,4.594737,14.526316,1.210526,19,10
